In [1]:
import sys
sys.path.append("/home/shkim/Bug-detecter-in-game/Bounce_Pygame_fixed")

In [2]:
import os
os.getcwd()

'/home/shkim/Bug-detecter-in-game/Bounce_Pygame_fixed'

In [1]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
from main import Game
class MyGameEnv(gym.Env):
    def __init__(self, render_mode=False, log_mode=False, bug_mode=False):
        super().__init__()
        self.render_mode = render_mode
        self.log_mode = log_mode
        self.bug_mode = bug_mode
        self.game = Game(render_mode=render_mode, log_mode=log_mode, bug_mode=bug_mode)

        self.action_space = spaces.Discrete(3)
        self.observation_space = spaces.Box(
            low=np.array([0.0, 0.0, -1.0, -1.0], dtype=np.float32),
            high=np.array([1.0, 1.0, 1.0, 1.0], dtype=np.float32),
            dtype=np.float32
        )

    def reset(self, seed=None, options=None):
        obs = self.game.reset()
        return np.array(obs, dtype=np.float32), {}

    def step(self, action):
        obs, reward, done, info = self.game.step(action)
        return np.array(obs, dtype=np.float32), reward, done, False, info

    def render(self):
        if self.render_mode:
            self.game.draw()

    def close(self):
        self.game.quitgame()


In [4]:
from stable_baselines3 import DQN
from stable_baselines3.common.env_checker import check_env

env = MyGameEnv()



In [ ]:
check_env(env, warn=True)  # 환경이 gym 스타일로 잘 되어 있는지 체크

In [5]:
model = DQN("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


KeyboardInterrupt: 

In [6]:
env.close()

SystemExit: 

/home/shkim/miniconda3/envs/bug_detector/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
obs, _ = env.reset()
done = False
while not done:
    action, _states = model.predict(obs)
    obs, reward, done, truncated, info = env.step(action)
    env.render()
    pg.time.delay(16)